In [2]:
# Imports using Sklearn make shortcut functions
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder
from sklearn.pipeline import make_pipeline

from xgboost import XGBRegressor

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

# Imports
import os
from google.cloud import bigquery

In [3]:
# Environment variables
gcp_project_id = os.environ['GCP_PROJECT']
gcp_service_account_key = os.environ['GCP_SERVICE_ACCOUNT_KEY']
bq_source_dataset = os.environ['BQ_SOURCE_DATASET']
bq_cleaned_dataset = os.environ['BQ_CLEANED_DATASET']

In [4]:
# Import data from BQ and set data to X
# Initialize a BigQuery client using the service account JSON file
bq_client = bigquery.Client(project=gcp_project_id).from_service_account_json(gcp_service_account_key)

# Setting the table we want from the source dataset
select_table = 'cleaned_full_polls_combined_national_results_2004_2019'

# SQL query for querying Big Query and fetching entire table
query = f"""
    SELECT *
    FROM `{gcp_project_id}.{bq_cleaned_dataset}.{select_table}`
"""

In [5]:
# Use BQ client to create DF from the selected table
data = bq_client.query(query).to_dataframe()

/Users/nieksonneveld/.pyenv/versions/3.10.6/envs/election_predictor/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [6]:
# Replace any found NaN values with 0
data.replace(np.nan, 0, inplace=True)

# Add a field 'poll length' that shows number of days the poll was held for
data['poll_length'] = pd.to_datetime(data.enddate) - pd.to_datetime(data.startdate)
data['poll_length'] = data['poll_length'].dt.days

# Divide forecasts by 100 to create values between 0-1
for column in ['BRX_FC', 'CON_FC', 'GRE_FC', 'LAB_FC', 'LIB_FC', 'NAT_FC', 'OTH_FC', 'PLC_FC', 'SNP_FC', 'UKI_FC']:
    data[column] = data[column] / 100

    # Divide actuals by 100 to create values between 0-1
for column in ['BRX_ACT', 'CON_ACT', 'GRE_ACT', 'LIB_ACT', 'LAB_ACT', 'NAT_ACT', 'PLC_ACT', 'SNP_ACT', 'UKI_ACT', 'OTH_PERCENTAGE']:
    data[column] = data[column] / 100

data.sort_values('enddate', inplace=True)

data.reset_index(inplace=True)

In [7]:
# Load Trends csvs from raw_data folder
trends1 = pd.read_csv('/Users/nieksonneveld/code/nieksonneveld/election-predictor/raw_data/LAB_CON_LIB_GRE_BRX_Trends_2004_now.csv')
trends2 = pd.read_csv('/Users/nieksonneveld/code/nieksonneveld/election-predictor/raw_data/LAB_PLC_SNP_UKI_NAT_Trends_2004_now.csv')

In [8]:
# Drop duplicate Labour column
trends2.drop(columns='Labour Party: (United Kingdom)',inplace=True)

In [9]:
# Join two trends datasets on 'Month'
trends_merged = pd.merge(trends1,trends2,how='left',on='Month')

In [10]:
# Convert 'Month' field to pd.datetime64[ns] format for joining with polling data
trends_merged['Month'] = pd.to_datetime(trends_merged['Month'])

In [11]:
# Replace <1 values with 0.5 value
trends_merged.replace('<1', 0.5,inplace=True)

In [12]:
# Converting objects (pd.Series) to int dtypes
trends_merged = trends_merged.astype({'Green Party: (United Kingdom)':'int','Reform UK: (United Kingdom)':'int',\
    'Plaid Cymru: (United Kingdom)':'int','Scottish National Party: (United Kingdom)':'int',\
        'UK Independence Party: (United Kingdom)':'int','British National Party: (United Kingdom)':'int'})

In [13]:
# Rename columns to align them more easily with _FC and _ACT columns
trends_merged.rename(columns={'Labour Party: (United Kingdom)': 'LAB_trends',
        'Conservative Party: (United Kingdom)': 'CON_trends',
        'Liberal Democrats: (United Kingdom)': 'LIB_trends',
        'Green Party: (United Kingdom)': 'GRE_trends',
        'Reform UK: (United Kingdom)': 'BRX_trends',
        'Plaid Cymru: (United Kingdom)': 'PLC_trends',
        'Scottish National Party: (United Kingdom)': 'SNP_trends',
        'UK Independence Party: (United Kingdom)': 'UKI_trends',
        'British National Party: (United Kingdom)': 'NAT_trends'}
        ,inplace=True)

In [15]:
# Create poll enddate field with YYYY-MM format, so we can join with Trends data
data['enddate_year_month'] = pd.to_datetime(data['enddate']).dt.to_period('M')

In [20]:
# Convert Period datatype to str and then to datetime64
data.enddate_year_month = pd.to_datetime(data.enddate_year_month.astype('str'))

In [22]:
# Merge data with trends data, to get a df with polls, election results and trends
data_trends = pd.merge(data,trends_merged,how='left',left_on='enddate_year_month',right_on='Month')

In [23]:
data_trends

,index,startdate,enddate,pollster,samplesize,rating,next_elec_date,days_to_elec,BRX_FC,CON_FC,...,Month,LAB_trends,CON_trends,LIB_trends,GRE_trends,BRX_trends,PLC_trends,SNP_trends,UKI_trends,NAT_trends
0,1298,2004-01-02,2004-01-04,Populus,566,D+,2005-05-05,489,0.00,0.35,...,2004-01-01,8.0,6.0,3.0,1.0,0.0,0.0,1.0,0.0,2.0
1,1277,2004-01-16,2004-01-18,ICM,1007,D+,2005-05-05,475,0.00,0.34,...,2004-01-01,8.0,6.0,3.0,1.0,0.0,0.0,1.0,0.0,2.0
2,1299,2004-02-06,2004-02-08,Populus,580,D+,2005-05-05,454,0.00,0.31,...,2004-02-01,8.0,6.0,3.0,1.0,0.0,0.0,1.0,0.0,3.0
3,1278,2004-02-20,2004-02-22,ICM,1006,D+,2005-05-05,440,0.00,0.34,...,2004-02-01,8.0,6.0,3.0,1.0,0.0,0.0,1.0,0.0,3.0
4,1300,2004-03-05,2004-03-07,Populus,573,D+,2005-05-05,426,0.00,0.34,...,2004-03-01,8.0,5.0,3.0,1.0,0.0,0.0,1.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255,561,2019-12-10,2019-12-11,Opinium,3005,A-,2019-12-12,2,0.02,0.45,...,2019-12-01,80.0,56.0,16.0,6.0,6.0,1.0,11.0,2.0,1.0
3256,619,2019-12-10,2019-12-11,Panelbase,3174,A-,2019-12-12,2,0.04,0.43,...,2019-12-01,80.0,56.0,16.0,6.0,6.0,1.0,11.0,2.0,1.0
3257,605,2019-12-09,2019-12-11,IpsosMORI,2213,A-,2019-12-12,3,0.02,0.44,...,2019-12-01,80.0,56.0,16.0,6.0,6.0,1.0,11.0,2.0,1.0
3258,361,2019-12-09,2019-12-11,Kantar,2815,B+,2019-12-12,3,0.03,0.44,...,2019-12-01,80.0,56.0,16.0,6.0,6.0,1.0,11.0,2.0,1.0
